In [1]:
from utils import *
from variables import *
from utils import parse_excel, read_pickle
from glob import glob

In [2]:
dict_auswahl_prefix = {0 : "", 1 : "STOXX_"}


list_files = glob("./excel_files/*.xlsm")
list_error_files = []

for excel_path in list_files[-1:]:

    if "stoxx" in excel_path.lower():
        auswahl = 1
    else:
        auswahl = 0

    auswahl, ZentralKurs, volatility, InterestRate, tage_bis_verfall, nbd_dict, dict_prod_bus, stock_price, expiry, expiry_1, heute, list_email_send_selection, future_date_col = parse_excel(auswahl, excel_path)
    


    Summery_df, HedgeBedarf_df, HedgeBedarf1_df, Ueberhaenge_df, delta= hedge(auswahl, ZentralKurs, volatility, InterestRate, tage_bis_verfall, dict_prod_bus, stock_price, expiry, expiry_1, heute)

    summery_cols = ['Basis', 'Änderung', 'heute', 'last_day']
    file_path = os.path.join(current_results_path, f"{heute.strftime('%Y_%m_%d')}_{dict_index_stock[auswahl]}.pdf")
    generate_pdfs(auswahl, Summery_df, HedgeBedarf_df, HedgeBedarf1_df, stock_price, heute, nbd_dict, tage_bis_verfall, delta, expiry, expiry_1, file_path)
    Summery_df = Summery_df[summery_cols]


    HedgeBedarf_excel = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}HedgeBedarf",skiprows=3).rename(columns = {"Unnamed: 0" : "Basis", "Unnamed: 2" : "HedgeSum"})[['Basis', 'HedgeSum']]


    HedgeBedarf1_excel = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}HedgeBedarf+01",skiprows=3).rename(columns = {"Unnamed: 0" : "Basis", "Unnamed: 2" : "HedgeSum"})[['Basis', 'HedgeSum']]


    Summery_excel = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Summery",skiprows=8).iloc[:,[0,1,2,3]]
    Summery_excel.columns = summery_cols
    Summery_excel = Summery_excel[pd.to_numeric(Summery_excel.Basis, errors= 'coerce').notna()]


    compare1_df = HedgeBedarf_df[['Basis', 'HedgeSum']].merge(HedgeBedarf_excel, on = 'Basis',suffixes=('_py', '_excel'))
    # Biggest difference between python and excel
    (compare1_df.HedgeSum_py - compare1_df.HedgeSum_excel).abs().max()
    compare1_df['dif'] = (compare1_df.HedgeSum_py - compare1_df.HedgeSum_excel).abs()
    max_dif1  = compare1_df.dif.abs().max()

    compare2_df = HedgeBedarf1_df[['Basis', 'HedgeSum']].merge(HedgeBedarf1_excel, on = 'Basis',suffixes=('_py', '_excel'))
    # Biggest difference between python and excel
    (compare2_df.HedgeSum_py - compare2_df.HedgeSum_excel).abs().max()
    compare2_df['dif'] = (compare2_df.HedgeSum_py - compare2_df.HedgeSum_excel).abs()
    max_dif2  = compare2_df.dif.abs().max()


    compare3_df = Summery_excel.merge(Summery_df, on = 'Basis', suffixes = ('_excel', '_python'))

    list_max = []
    for col in Summery_df.columns[1:]:
        list_max.append((compare3_df[f'{col}_excel'] - compare3_df[f'{col}_python']).abs().max())

    max_dif3 = max(list_max)


    if max(max_dif1, max_dif2, max_dif3) > 1e-6:
        list_error_files.append(excel_path)
        raise ValueError("Excel and python results are not the same")
    print("Both files are the same.")
print(list_error_files)

c:\Users\Vinicius\anaconda3\envs\eurex\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Vinicius\anaconda3\envs\eurex\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


busdate_idx = 0   | productdate_idx = 0
busdate_idx = 1   | productdate_idx = 0
busdate_idx = 0   | productdate_idx = 1
busdate_idx = 1   | productdate_idx = 1
heute = 11/09/2023
tage_bis_verfall = 4
expiry = 2023-09-15 00:00:00
expiry_1 = 2023-10-20 00:00:00
stock_price = 15735.5
ZentralKurs = 16000
volatility = 0.1531
InterestRate = 0.038
Minkurs = 15000.0
Maxkurs = 17000.0
Schritte = 40
volatility_Laufzeit = 60
Tage = 4
Tage_1 = 39
PDF has been generated.
Both files are the same.
[]


In [5]:
auswahl = 0
result = read_pickle(os.path.join(temp_results_path,f'{dict_index_stock[auswahl]}.pickle'))

auswahl, ZentralKurs, volatility, InterestRate, tage_bis_verfall, nbd_dict, dict_prod_bus, stock_price, expiry, expiry_1, heute, list_email_send_selection, future_date_col = result

Summery_df, HedgeBedarf_df, HedgeBedarf1_df, Ueberhaenge_df, delta= hedge(auswahl, ZentralKurs, volatility, InterestRate, tage_bis_verfall, dict_prod_bus, stock_price, expiry, expiry_1, heute)

file_path = os.path.join(current_results_path, f"{heute.strftime('%Y_%m_%d')}_{dict_index_stock[auswahl]}.pdf")

generate_pdfs(auswahl, Summery_df, HedgeBedarf_df, HedgeBedarf1_df, stock_price, heute, nbd_dict, tage_bis_verfall, delta, expiry, expiry_1, file_path)

FileNotFoundError: [Errno 2] No such file or directory: './Results/temp\\DAX.pickle'